Assignment_1 AWS Pipeline Architecture-Team 6


In [0]:
###INFO7374: Digital Marketing Analytics:
###LAB: AWS Mobile Gaming Analytics Pipeline
###Date: 06/28/2019
###Instructor: Prof. Srikanth Krishnamurthy
###TA: Mr. Pramod Nagare and Mr. Syed Haroon

In [0]:
# Architectural Components:
# S3 Bucket: https://aws.amazon.com/s3/
# Kinesis Data Stream: https://aws.amazon.com/kinesis/data-streams/
# Kinesis Delivery Stream: https://aws.amazon.com/kinesis/data-firehose/
# Redshift Cluster: https://aws.amazon.com/redshift/
# IAM Policy: https://aws.amazon.com/iam/
# IAM Role: https://aws.amazon.com/iam/

<img src="AWS_GAP_Architecture_New.jpg">

In [0]:
!pip install boto3

    100% |████████████████████████████████| 133kB 5.6MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 9.2MB/s eta 0:00:01
    100% |████████████████████████████████| 5.6MB 5.4MB/s eta 0:00:01


In [0]:
!pip install psycopg2

    100% |████████████████████████████████| 378kB 11.8MB/s ta 0:00:01
    Complete output from command python setup.py egg_info:
    running egg_info
    creating pip-egg-info/psycopg2.egg-info
    writing pip-egg-info/psycopg2.egg-info/PKG-INFO
    writing dependency_links to pip-egg-info/psycopg2.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/psycopg2.egg-info/top_level.txt
    writing manifest file 'pip-egg-info/psycopg2.egg-info/SOURCES.txt'
    
    Error: pg_config executable not found.
    
    pg_config is required to build psycopg2 from source.  Please add the directory
    containing pg_config to the $PATH or specify the full executable path with the
    option:
    
        python setup.py build_ext --pg-config /path/to/pg_config build ...
    
    or with the pg_config option in 'setup.cfg'.
    
    If you prefer to avoid building psycopg2 from source, please install the PyPI
    'psycopg2-binary' package instead.
    
    For further information

In [0]:
#Importing all required libraries:
#pip install boto3
#pip install psycopg2

import boto3
import json
import psycopg2
import datetime

In [0]:
#Setting up the AWS Access keys:
#NOTE: DO NOT EXPOSE THE ACCESS & SECRET KEYS

config_file = open(r'Config.json')
config_file = json.loads(config_file.read())

access_key = config_file['access_key']
secret_access_key = config_file['secret_access_key']


In [0]:
#Configuration Parameters:

bucket_name = "info7374s3assignment1team6"#------------------------------------S3: Name of the creating AWS S3 Bucket

policy_name = "info7374policyassignment1team6"#----------------------------------IAM: Access policy name
iam_role_name = "info7374roleassignment1team6"#----------------------------------IAM: IAM Role for the architecural access

db_name = "info7374dbassignment1team6"#-------------------------------------Redshift: Database Name for gaming data
cluster_identifier = "info7374clusterassignment1team6"#---------------------Redshift: Redshift Cluster Name
master_username = "root"#----------------------------------------Redshift: Admin Username
master_password = "Info7374gap"#---------------------------------Redshift: Admin Password
node_type = "dc2.large"#-----------------------------------------Redshift: Cluster configuration
cluster_type = "single-node"#------------------------------------Redshift: Cluster Type
availability_zone = "us-east-1a"#--------------------------------Redshift: Cluster Availability Zone
table_name = "info7374_game"#------------------------------------Redshift: Database table name
kinesis_data_stream_name = "info7374datastreamassignment1team6"#-------------Kinesis: Data Stream Name
shard_count = 100#------------------------------------------------Kinesis: Data Stream Shard Count

log_group_name = "info7374loggroupassignment1team6"#------------------------CloudWatch: Log Group Name
log_stream_name = "info7374logstreamassignment1team6"#------------------------CloudWatch: Log Group Name
delivery_stream_name = "info7374deliverystreamassignment1team6"#--------------Kinesis: Delivery Stream Name
stream_type = "KinesisStreamAsSource"#-----------------------------Kinesis: Delivery Stream Type


In [0]:
#Creating AWS S3 Bucket:

s3_client = boto3.client('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)

response_s3 = s3_client.create_bucket(ACL="private", Bucket=bucket_name)
print(response_s3)

{'ResponseMetadata': {'RequestId': '71CF188C00A0F21E', 'HostId': 'QADcwPiOSxsb4ruKzEWy3FNGNEKeMMFd/JC7ZhVG/qx1zPoJdJx6XsY86NpRih5rIi2RAJLpXtk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'QADcwPiOSxsb4ruKzEWy3FNGNEKeMMFd/JC7ZhVG/qx1zPoJdJx6XsY86NpRih5rIi2RAJLpXtk=', 'x-amz-request-id': '71CF188C00A0F21E', 'date': 'Thu, 11 Jul 2019 13:59:44 GMT', 'location': '/info7374s3assignment1team6', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Location': '/info7374s3assignment1team6'}


In [0]:
bucket_arn = "arn:aws:s3:::"+bucket_name#---------------------------S3: Bucker ARN
print(bucket_arn)

arn:aws:s3:::info7374s3assignment1team6


In [0]:
#Creating IAM Policy for Architecural access:

iam_client = boto3.client('iam', aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)


In [0]:

policy_details = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "glue:GetTableVersions"
            ],
            "Resource": "*"
        },
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject"
            ],
            "Resource": "*"
        },
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Resource": "*"
        },
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "logs:PutLogEvents"
            ],
            "Resource": "*"
        },
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:GetShardIterator",
                "kinesis:GetRecords"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "kms:Decrypt"
            ],
            "Resource": "*"
        }
    ]
}

In [0]:
response_iam_policy = iam_client.create_policy(PolicyName=policy_name, PolicyDocument=json.dumps(policy_details))
print(response_iam_policy)

{'Policy': {'PolicyName': 'info7374policyassignment1team6', 'PolicyId': 'ANPATOYZ5O3I7HOLJXZZW', 'Arn': 'arn:aws:iam::237888173777:policy/info7374policyassignment1team6', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2019, 7, 11, 14, 2, 7, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2019, 7, 11, 14, 2, 7, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '786422d6-a3e4-11e9-b4bc-116c7ed14b88', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '786422d6-a3e4-11e9-b4bc-116c7ed14b88', 'content-type': 'text/xml', 'content-length': '795', 'date': 'Thu, 11 Jul 2019 14:02:07 GMT'}, 'RetryAttempts': 0}}


In [0]:
policy_arn = response_iam_policy['Policy']['Arn']#------------------------IAM: Policy ID for assignment
print(policy_arn)

arn:aws:iam::237888173777:policy/info7374policyassignment1team6


In [0]:
#Creating IAM Role for Architectural access:

assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "firehose.amazonaws.com"
      },
      "Action": "sts:AssumeRole",
      "Condition": {
        "StringEquals": {
          "sts:ExternalId": "237888173777"
        }
      }
       
    }
  ]
}

response_iam_role = iam_client.create_role(RoleName=iam_role_name, AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc))

print(response_iam_role)

{'Role': {'Path': '/', 'RoleName': 'info7374roleassignment1team6', 'RoleId': 'AROATOYZ5O3I6EM26DGT7', 'Arn': 'arn:aws:iam::237888173777:role/info7374roleassignment1team6', 'CreateDate': datetime.datetime(2019, 7, 11, 14, 2, 13, tzinfo=tzlocal()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Sid': '', 'Effect': 'Allow', 'Principal': {'Service': 'firehose.amazonaws.com'}, 'Action': 'sts:AssumeRole', 'Condition': {'StringEquals': {'sts:ExternalId': '237888173777'}}}]}}, 'ResponseMetadata': {'RequestId': '7bf2eecb-a3e4-11e9-b4bc-116c7ed14b88', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7bf2eecb-a3e4-11e9-b4bc-116c7ed14b88', 'content-type': 'text/xml', 'content-length': '948', 'date': 'Thu, 11 Jul 2019 14:02:13 GMT'}, 'RetryAttempts': 0}}


In [0]:
role_arn = response_iam_role['Role']['Arn']#----------------------------IAM: Role ARN

In [0]:
#Attaching a Policy to a Role:

response_iam_role_policy_attach = iam_client.attach_role_policy(PolicyArn=policy_arn,RoleName=iam_role_name)
print(response_iam_role_policy_attach)

{'ResponseMetadata': {'RequestId': '7e7ad22b-a3e4-11e9-b4bc-116c7ed14b88', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7e7ad22b-a3e4-11e9-b4bc-116c7ed14b88', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 11 Jul 2019 14:02:17 GMT'}, 'RetryAttempts': 0}}


In [18]:
#Creating AWS Redshift Cluster:

redshift_client = boto3.client('redshift',region_name="us-east-1", aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)

response_redshift = redshift_client.create_cluster(
    DBName=db_name,
    ClusterIdentifier=cluster_identifier,
    ClusterType=cluster_type,
    NodeType=node_type,
    MasterUsername=master_username,
    MasterUserPassword=master_password,
    AvailabilityZone=availability_zone,
    IamRoles=[role_arn],
    PubliclyAccessible=True)

print(response_redshift)

NameError: ignored

In [0]:
#Note: Wait for 5 min to get AWS Redshift cluster to setup as we need to create a Database table.

In [21]:
#Describe AWS Redshift Cluster to get an endpoint:

response_redshift_desc = redshift_client.describe_clusters(ClusterIdentifier=cluster_identifier)
print(response_redshift_desc)

{'Clusters': [{'ClusterIdentifier': 'info7374clusterassignment1team6', 'NodeType': 'dc2.large', 'ClusterStatus': 'available', 'ClusterAvailabilityStatus': 'Available', 'MasterUsername': 'root', 'DBName': 'info7374dbassignment1team6', 'Endpoint': {'Address': 'info7374clusterassignment1team6.c76nrcvogw31.us-east-1.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2019, 7, 11, 14, 6, 37, 65000, tzinfo=tzlocal()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-7d4d2226', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-ad5030d7', 'AvailabilityZone': 'us-east-1a', 'PreferredMaintenanceWindow': 'mon:09:30-mon:10:00', 'PendingModifiedValues': {}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 1, 'Public

In [0]:
hostname = response_redshift_desc['Clusters'][0]['Endpoint']['Address']#----------------Redshift: Hostname for database
port_number = response_redshift_desc['Clusters'][0]['Endpoint']['Port']#----------------Redshift: Port Number for databse

In [0]:
#Creating Database table on Redshift:

con = psycopg2.connect(dbname= db_name, host= hostname, port= port_number, user= master_username, password= master_password)

con.set_session(autocommit=True)
cur = con.cursor()

In [32]:
#Example: https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_TABLE_examples.html

query = "create table Products (\n"
query+= "product_id varchar(200) not null,\n"
query+= "product_name varchar(200) not null,\n"
query+= "category varchar(200) not null);"

print(query)

create table Products (
product_id varchar(200) not null,
product_name varchar(200) not null,
category varchar(200) not null);


In [0]:
cur.execute(query)

In [0]:
copy products
query = "create table Products (\n"
query+= "product_id varchar(200) not null,\n"
query+= "product_name varchar(200) not null,\n"
query+= "category varchar(200) not null);"

print(query)

SyntaxError: ignored

In [0]:
copy products from 's3://info7374s3assignment1team6/JSON/Products_data.json' 
credentials 'aws_access_key_id=<access_key>;aws_secret_access_key=<secret_access_key>' 
options;

In [53]:
query = "create table sales (\n"
query+= "Order_id integer not null,\n"
query+= "Customer_Category varchar(200) not null,\n"
query+= "Customer_ID varchar(200) not null,\n"
query+= "Customer_Name varchar(200) not null,\n"
query+= "Order_Date date not null,\n"
query+= "Promotion_id varchar(200) not null,\n"
query+= "product_id varchar(200) not null,\n"
query+= "Price decimal(8,2) not null,\n"
query+= "quantity decimal(8,2) not null,\n"
query+= "total_amount decimal(8,2) not null);"

print(query)

create table sales (
Order_id integer not null,
Customer_Category varchar(200) not null,
Customer_ID varchar(200) not null,
Customer_Name varchar(200) not null,
Order_Date date not null,
Promotion_id varchar(200) not null,
product_id varchar(200) not null,
Price decimal(8,2) not null,
quantity decimal(8,2) not null,
total_amount decimal(8,2) not null);


In [0]:
cur.execute(query)

In [55]:
query = "create table Customers (\n"
query+= "Customer_ID integer not null,\n"
query+= "Company_Name varchar(200) not null,\n"
query+= "Region varchar(200) not null,\n"
query+= "State varchar(200) not null,\n"
query+= "City varchar(200) not null,\n"
query+= "Postal_Code integer not null,\n"
query+= "Cusomter_Category varchar(200) not null);"

print(query)



create table Customers (
Customer_ID integer not null,
Company_Name varchar(200) not null,
Region varchar(200) not null,
State varchar(200) not null,
City varchar(200) not null,
Postal_Code integer not null,
Cusomter_Category varchar(200) not null);


In [0]:
cur.execute(query)

In [44]:
query = "create table Products_pricing (\n"
query+= "product_id varchar(200) not null,\n"
query+= "cost_price decimal(8,2) not null,\n"
query+= "selling_price decimal(8,2) not null);"

print(query)


create table Products_pricing (
product_id varchar(200) not null,
cost_price decimal(8,2) not null,
selling_price decimal(8,2) not null);


In [0]:
cur.execute(query)

In [46]:
query = "create table Promotions (\n"
query+= "Promotion_id varchar(200) not null,\n"
query+= "Promotion_Name varchar(200) not null,\n"
query+= "Promotion_Start_Date date not null,\n"
query+= "Promotion_End_Date date not null);"

print(query)

create table Promotions (
Promotion_id varchar(200) not null,
Promotion_Name varchar(200) not null,
Promotion_Start_Date date not null,
Promotion_End_Date date not null);


In [0]:
cur.execute(query)

In [57]:
#Creating Kinesis Stream:

kinesis_client = boto3.client('kinesis',region_name="us-east-1", aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)

response_kinesis_data_stream = kinesis_client.create_stream(StreamName=kinesis_data_stream_name,ShardCount=shard_count)
print(response_kinesis_data_stream)

{'ResponseMetadata': {'RequestId': 'fe2f7b56-5365-a60f-ab8c-6eeca00a4fec', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fe2f7b56-5365-a60f-ab8c-6eeca00a4fec', 'x-amz-id-2': 'qtG+wEW6RpXyBAkeFsrD10OATgH/bpV+cTU+SexJuuZlCmEzltMpWWDpSAoYR4UTuPVvU6/8fRua5C8CGpu+7uZmGkLpBmQz', 'date': 'Fri, 12 Jul 2019 23:10:50 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0'}, 'RetryAttempts': 0}}


In [58]:
response_kinesis_data_stream_desc = kinesis_client.describe_stream(StreamName=kinesis_data_stream_name)
print(response_kinesis_data_stream_desc)

{'StreamDescription': {'StreamName': 'info7374datastreamassignment1team6', 'StreamARN': 'arn:aws:kinesis:us-east-1:237888173777:stream/info7374datastreamassignment1team6', 'StreamStatus': 'CREATING', 'Shards': [], 'HasMoreShards': False, 'RetentionPeriodHours': 24, 'StreamCreationTimestamp': datetime.datetime(2019, 7, 12, 23, 10, 49, tzinfo=tzlocal()), 'EnhancedMonitoring': [{'ShardLevelMetrics': []}], 'EncryptionType': 'NONE'}, 'ResponseMetadata': {'RequestId': 'f761f290-1b92-15ef-a2c2-e72be8fdfc0c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f761f290-1b92-15ef-a2c2-e72be8fdfc0c', 'x-amz-id-2': 'ffiB+/oIxbpY9He/aepu2xEaNapeoSX6l3kKQ28mLc5Ke83eVwyDfou6Hc33VvPHxHqK7D+4aDmjAdFZUBZcP9q8hEywgQbl', 'date': 'Fri, 12 Jul 2019 23:10:51 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '366'}, 'RetryAttempts': 0}}


In [59]:
kinesis_stream_arn = response_kinesis_data_stream_desc['StreamDescription']['StreamARN']#-----------------------Kinesis: Datastream ARN
print(kinesis_stream_arn)

arn:aws:kinesis:us-east-1:237888173777:stream/info7374datastreamassignment1team6


In [60]:
#Creating Kinesis Delivery Stream: Firehose

firehose_client = boto3.client('firehose',region_name="us-east-1", aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)

cluster_jdbc_url = "jdbc:redshift://"+hostname+":"+str(port_number)+"/"+db_name
print(cluster_jdbc_url)

jdbc:redshift://info7374clusterassignment1team6.c76nrcvogw31.us-east-1.redshift.amazonaws.com:5439/info7374dbassignment1team6


In [66]:
response_firehose = firehose_client.create_delivery_stream(
    DeliveryStreamName = delivery_stream_name,
    DeliveryStreamType = stream_type,
    KinesisStreamSourceConfiguration={
        'KinesisStreamARN': kinesis_stream_arn,
        'RoleARN': 'arn:aws:iam::237888173777:role/info7374roleassignment1team6'
    },
    RedshiftDestinationConfiguration={
        'RoleARN': 'arn:aws:iam::237888173777:role/info7374roleassignment1team6',
        'ClusterJDBCURL': cluster_jdbc_url,
        'CopyCommand': {
            'DataTableName': 'products',
            'DataTableColumns': 'Product_ID, Product_Name, Category',
            'CopyOptions': "json 'auto'"
        },
        'Username': master_username,
        'Password': master_password,
        'S3Configuration': {
            'RoleARN': 'arn:aws:iam::237888173777:role/info7374roleassignment1team6',
            'BucketARN': 'arn:aws:s3:::info7374s3assignment1team6',
            'BufferingHints': {
                'IntervalInSeconds': 60
            }
        }
    })

print(response_firehose)

{'DeliveryStreamARN': 'arn:aws:firehose:us-east-1:237888173777:deliverystream/info7374deliverystreamassignment1team6', 'ResponseMetadata': {'RequestId': 'd5191935-f78c-9748-80ba-0375abaf75ce', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd5191935-f78c-9748-80ba-0375abaf75ce', 'x-amz-id-2': 'QlhxJbt4Y0JySVuIv/DAE7pfcOyVUK2O20KM/LWNNNq+pakfMk5exx0r+kHPg/MRGmo1FK422HiJaqdJb+uLtml7Awg5oiBE', 'content-type': 'application/x-amz-json-1.1', 'content-length': '117', 'date': 'Fri, 12 Jul 2019 23:30:08 GMT'}, 'RetryAttempts': 0}}


In [0]:
#Cogratulation!!! At this stage we have successfully created our Architacure

In [0]:
#Real-time data streaming:

kinesis_delivery_client = boto3.client('kinesis', region_name="us-east-1", aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)

In [0]:
#My Data

import pandas as pd



In [76]:
#Reading the csv file
data_file = r'Products.csv'
data = pd.read_csv(data_file)
data.head()

,Product ID,Product Name,Category
0,P1,725C2 BARE CHASSIS,ARTICULATED TRUCKS
1,P2,730C2 BARE CHASSIS,ARTICULATED TRUCKS
2,P3,740 EJ,ARTICULATED TRUCKS
3,P4,415F2,ARTICULATED TRUCKS
4,P5,415F2 IL,ARTICULATED TRUCKS


In [0]:
data = data.head(5)

In [78]:
data

,Product ID,Product Name,Category
0,P1,725C2 BARE CHASSIS,ARTICULATED TRUCKS
1,P2,730C2 BARE CHASSIS,ARTICULATED TRUCKS
2,P3,740 EJ,ARTICULATED TRUCKS
3,P4,415F2,ARTICULATED TRUCKS
4,P5,415F2 IL,ARTICULATED TRUCKS


In [79]:
data.columns

Index(['Product ID', 'Product Name', 'Category'], dtype='object')

In [84]:
for i in range(0,data.shape[0]):
  product_id = data['Product ID'][i]
  product_name = data['Product Name'][i]
  category = data['Category'][i]
  #Product_ID, Product_Name, Category
  data_row = {"Product_ID":product_id,"Product_Name":product_name,"Category":category}
  data_row = json.dumps(data_row)
  data_row = bytes(data_row, 'utf-8')
  formatted_records = [{'PartitionKey': "pk",'Data': data_row}]
  print(formatted_records)
  delivery_stream_name = kinesis_data_stream_name
  response = kinesis_delivery_client.put_records(StreamName=delivery_stream_name, Records=formatted_records)

[{'PartitionKey': 'pk', 'Data': b'{"Product_ID": "P1", "Product_Name": "725C2 BARE CHASSIS", "Category": "ARTICULATED TRUCKS"}'}]
[{'PartitionKey': 'pk', 'Data': b'{"Product_ID": "P2", "Product_Name": "730C2 BARE CHASSIS", "Category": "ARTICULATED TRUCKS"}'}]
[{'PartitionKey': 'pk', 'Data': b'{"Product_ID": "P3", "Product_Name": "740 EJ", "Category": "ARTICULATED TRUCKS"}'}]
[{'PartitionKey': 'pk', 'Data': b'{"Product_ID": "P4", "Product_Name": "415F2", "Category": "ARTICULATED TRUCKS"}'}]
[{'PartitionKey': 'pk', 'Data': b'{"Product_ID": "P5", "Product_Name": "415F2 IL", "Category": "ARTICULATED TRUCKS"}'}]


In [85]:
response

{'FailedRecordCount': 0,
 'Records': [{'SequenceNumber': '49597544726370028189063812412712857130553560848117268658',
   'ShardId': 'shardId-000000000011'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '146',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 12 Jul 2019 23:44:25 GMT',
   'x-amz-id-2': 'GYM7ukF3PUmvIxHpG5FSdOObDr/J0NWD9cwgjkUB8xqgI3kzDUwGJRxsUzysHprzZoAP29jSXKf6sHYekd+7vYj9kM0NwU3i',
   'x-amzn-requestid': 'fd838f59-c147-115d-a820-92c0314be0db'},
  'HTTPStatusCode': 200,
  'RequestId': 'fd838f59-c147-115d-a820-92c0314be0db',
  'RetryAttempts': 0}}

In [0]:
#Now we will demolish the complete architecture:

#Delete Redshift Cluster:
response_delete_redshift = redshift_client.delete_cluster(
    ClusterIdentifier=cluster_identifier,
    SkipFinalClusterSnapshot=True
)

print(response_delete_redshift)

{'Cluster': {'ClusterIdentifier': 'info7374clusterassignment1team6', 'NodeType': 'dc2.large', 'ClusterStatus': 'deleting', 'MasterUsername': 'root', 'DBName': 'info7374dbassignment1team6', 'Endpoint': {'Address': 'info7374clusterassignment1team6.c76nrcvogw31.us-east-1.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2019, 7, 5, 20, 53, 17, 649000, tzinfo=tzlocal()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-7d4d2226', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-ad5030d7', 'AvailabilityZone': 'us-east-1a', 'PreferredMaintenanceWindow': 'fri:05:30-fri:06:00', 'PendingModifiedValues': {}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 1, 'PubliclyAccessible': True, 'Encrypted': False, 'Ta

In [0]:
#Delete Kinesis Delivery stream:
response_delete_firehose = firehose_client.delete_delivery_stream(DeliveryStreamName=delivery_stream_name)
print(response_delete_firehose)

{'ResponseMetadata': {'RequestId': 'f265a551-6d0d-813c-a7fc-640707b935de', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f265a551-6d0d-813c-a7fc-640707b935de', 'x-amz-id-2': 'xpdB5e+ZkGTJd61xO0kff6+DrZe3JsrpNXda5vzTlXWfnv3upWrM81CVQ303UAdRQ9vItH6AXwTzjTYjwmLpPoa2hZ5vdngt', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 05 Jul 2019 21:20:21 GMT'}, 'RetryAttempts': 0}}


In [0]:
#Delete Kinesis Data stream:
response_delete_data_stream = kinesis_client.delete_stream(StreamName=kinesis_data_stream_name,EnforceConsumerDeletion=True)
print(response_delete_data_stream)

{'ResponseMetadata': {'RequestId': 'dde266dd-bd64-b23e-887b-a78a9ebee03f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dde266dd-bd64-b23e-887b-a78a9ebee03f', 'x-amz-id-2': 'QfniCnWNQ/h4aupAlYE/Tf2gXnjxVFIPAZv6fm9kjO6A8qfkvCMY4r6naa85eVdtxBtxsWNaoY7g9Y5mD1MBalA5wge/HrSD', 'date': 'Fri, 5 Jul 2019 21:20:23 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0'}, 'RetryAttempts': 0}}


In [0]:
#Delete S3 Bucket:

#All of the Objects in a bucket must be deleted before the bucket itself can be deleted:

s3 = boto3.resource('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_access_key)
bucket = s3.Bucket(bucket_name)

for key in bucket.objects.all():
    key.delete()
bucket.delete()

print("Bucket deleted successfully!")

Bucket deleted successfully!


In [0]:
#Delete IAM Policy:



In [0]:
#Delete IAM Role:

response_detach_policy = iam_client.detach_role_policy(PolicyArn=policy_arn,RoleName=iam_role_name)
print(response_detach_policy)

response_delete_role = iam_client.delete_role(RoleName=iam_role_name)
print(response_delete_role)

{'ResponseMetadata': {'RequestId': 'b80ab56e-9f6a-11e9-a9e7-edf2f5f1aca7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b80ab56e-9f6a-11e9-a9e7-edf2f5f1aca7', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 05 Jul 2019 21:20:31 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'b811e192-9f6a-11e9-a9e7-edf2f5f1aca7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b811e192-9f6a-11e9-a9e7-edf2f5f1aca7', 'content-type': 'text/xml', 'content-length': '200', 'date': 'Fri, 05 Jul 2019 21:20:31 GMT'}, 'RetryAttempts': 0}}


In [0]:
#Delete IAM Policy:

response_delete_policy = iam_client.delete_policy(PolicyArn=policy_arn)
print(response_delete_policy)

{'ResponseMetadata': {'RequestId': 'b98aed42-9f6a-11e9-a9e7-edf2f5f1aca7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b98aed42-9f6a-11e9-a9e7-edf2f5f1aca7', 'content-type': 'text/xml', 'content-length': '204', 'date': 'Fri, 05 Jul 2019 21:20:33 GMT'}, 'RetryAttempts': 0}}
